In [1]:
import numpy as np
import math
from Crypto.Util.number import getPrime, inverse, bytes_to_long, long_to_bytes, GCD
import random

In [17]:
def iroot(k, n):
    u, s = n, n+1
    while u < s:
        s = u
        t = (k-1) * s + n // pow(s, k-1)
        u = t // k
    return s

In [3]:
def encrypt_rsa(m, e, N):
    assert m < N
    c = pow(m, e, N)
    return c

# Prerequisites

1. RSA + its prerequisites
2. Chinese remainder theorem
* https://en.wikipedia.org/wiki/Chinese_remainder_theorem
* https://www.youtube.com/watch?v=oKMYNKbFHBE&list=PLKXdxQAT3tCssgaWOy5vKXAR4WTPpRVYK&index=58

# Theory

**Task**: 

Given k moduli:    
$c_1 = m^e mod N_1 \\
c_2 = m^e mod N_2 \\
... \\
$ with $\gcd(N_i,N_j) = 1 \forall i \neq j$

find $m$

## CRT:


Let $m_{1}, m_{2}, \ldots, m_{k}$ be a collection of pairwise relatively prime integers. This means that  
$
\operatorname{gcd}\left(m_{i}, m_{j}\right)=1 \quad \text { for all } i \neq j
$  

Let $a_{1}, a_{2}, \ldots, a_{k}$ be arbitrary integers. 

Then the system of simultaneous congruences  
$  
x \equiv a_{1}\left(\bmod m_{1}\right), \\
x \equiv a_{2}\left(\bmod m_{2}\right), \\
\ldots\\
x \equiv a_{k}\left(\bmod m_{k}\right)
$  
has a solution $x=c .$  

Further, if $x=c$ and $x=c^{\prime}$ are both solutions, then
$
c \equiv c^{\prime} \quad\left(\bmod m_{1} m_{2} \cdots m_{k}\right)
$

Explanation: https://www.youtube.com/watch?v=aS57JCzJw_o

Condition to work: $k \geq e$

Since $m < N_i \forall i \in {1..k} => m^e < \Pi_{i=1}^k N_i$

Solve CRT for $m^e$

Extract the e-root and voila



## Hastad's findings:

If $c \equiv m^e \ mod \ N$ then we can find $m$ in polynomial time if $m < N^{1/e}$

# Code

## Crt

In [15]:
def solve_simple_eq(a, b, c, p):
    '''a + bx = c mod p'''
    c = c-a
    c = (c * inverse(b, p))% p
    return c%p

def crt(a_list, m_list):
    x = 0
    #starting values
    a = 0
    m = 1
    for i in range(len(m_list)):
        #general solution for the first equations x = a + m * y 
        #plug it in into the next equation and solve
        x = solve_simple_eq(a, m, a_list[i], m_list[i])
        
        a = a + m * x
        m = m * m_list[i]
        #print(a, m)        
    return a, m
        

In [10]:
N_list = []
c_list = []
e = 3
m = bytes_to_long(b'verylongsecret' * 10)
for _ in range(3):
    p = getPrime(1024)
    q = getPrime(1024)
    #assert GCD(e, (p-1)*(q-1)) == 1
    N = p * q
    c = pow(m, e, N)
    N_list.append(N)
    c_list.append(c)
    
    

In [18]:
m_decr_e, _ = crt(c_list, N_list)

In [19]:
m_decr = iroot(e, m_decr_e)
print(long_to_bytes(m_decr))

b'verylongsecretverylongsecretverylongsecretverylongsecretverylongsecretverylongsecretverylongsecretverylongsecretverylongsecretverylongsecret'


# Resources

* https://en.wikipedia.org/wiki/Coppersmith%27s_attack#H%C3%A5stad's_broadcast_attack
- https://koclab.cs.ucsb.edu/teaching/cren/project/2017/chennagiri.pdf